# Iperparametri e validazione del modello

Finora abbiamo visto in varie salse la ricetta base per applicare un modello di machine learning supervisionato:

1. Scegliere una classe di modello
2. Scegliere gli iperparametri del modello
3. Adattare il modello ai dati di addestramento
4. Utilizzare il modello per prevedere le etichette per i nuovi dati

I primi due elementi, la scelta del modello e la scelta degli iperparametri, sono forse la parte più importante per utilizzare questi strumenti e tecniche in modo efficace. Per fare una scelta informata, abbiamo bisogno di un modo per verificare che il nostro modello e i nostri iperparametri si adattino bene ai dati. Anche se questo può sembrare semplice, ci sono alcune insidie ​​​​che dobbiamo evitare per farlo in modo efficace, per questo voglio soffermarmi in maniera più approfondita su questi aspetti.

## Pensare alla validazione del modello

In linea di principio, la validazione del modello è molto semplice: dopo aver scelto un modello e i suoi iperparametri, possiamo stimarne l’efficacia applicandolo ad alcuni dati di training e confrontando la previsione con il valore noto.

Le sezioni seguenti mostrano innanzitutto un approccio ingenuo alla convalida del modello e il motivo per cui fallisce, prima di esplorare l'uso dei set di controllo e della convalida incrociata per una valutazione del modello più solida.

### Validazione del modello nel modo 'sbagliato'

Dimostriamo l'approccio ingenuo alla convalida utilizzando i dati Iris, che abbiamo visto nella sezione precedente. Inizieremo caricando i dati:

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

Successivamente scegliamo un modello e gli iperparametri. Qui utilizzeremo un classificatore kn_neighbors=1 , questo è un modello KNN molto semplice e intuitivo che dice "l'etichetta di un punto sconosciuto è la stessa dell'etichetta del suo punto di allenamento più vicino:"

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

Quindi addestriamo il modello e lo utilizziamo per prevedere le etichette per i dati che già conosciamo:

In [ ]:
model.fit(X, y)
y_model = model.predict(X)

Infine, calcoliamo la frazione di punti etichettati correttamente:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

Vediamo un punteggio di precisione pari a 1,0, che indica che il 100% dei punti sono stati etichettati correttamente dal nostro modello! Ma questo misura davvero la precisione attesa? Siamo davvero arrivati ​​a un modello che ci aspettiamo sia corretto il 100% delle volte?

Come avrai capito, la risposta è no. In effetti, questo approccio contiene un difetto fondamentale: addestra e valuta il modello sugli stessi dati . Inoltre, il modello KNN è uno stimatore basato su istanze che memorizza semplicemente i dati di addestramento e prevede le etichette confrontando i nuovi dati con questi punti memorizzati: tranne nei casi inventati, otterrà ogni volta una precisione del 100%!

### Convalida del modello nel modo giusto: set di controllo

Quindi cosa si può fare? Come abbiamo visto è possibile avere un'idea migliore delle prestazioni di un modello utilizzando il cosiddetto set di controllo : ovvero, tratteniamo alcuni sottoinsiemi di dati dall'addestramento del modello e quindi utilizziamo questo set di controllo per verificare le prestazioni del modello. Questa suddivisione può essere eseguita utilizzando l'utilità train_test_split in Scikit-Learn:

In [ ]:
from sklearn.model_selection import train_test_split
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)

Qui vediamo un risultato più ragionevole: il KNN è accurato al 90% circa su questo insieme. L'insieme di test è simile ai dati sconosciuti, perché il modello non lo ha "visto" prima.

### Convalida del modello tramite convalida incrociata

Uno svantaggio dell'utilizzo di un set di controlli per la convalida del modello è che abbiamo perso una parte dei nostri dati a causa dell'addestramento del modello. Nel caso precedente, metà del set di dati non contribuisce all'addestramento del modello! Questo non è ottimale e può causare problemi, soprattutto se il set iniziale di dati di addestramento è piccolo.

Un modo per risolvere questo problema è utilizzare la convalida incrociata, ovvero eseguire una sequenza di adattamenti in cui ciascun sottoinsieme di dati viene utilizzato sia come set di addestramento che come set di convalida. Visivamente, potrebbe assomigliare a questo:


![](dati/img/convalida%20incrociata.png)


Qui eseguiamo due prove di validazione, utilizzando alternativamente ciascuna metà dei dati come set di controllo. 
Utilizzando i dati divisi di prima, potremmo implementarlo in questo modo:

In [ ]:
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

Ciò che ne risulta sono due punteggi di accuratezza, che potremmo combinare (ad esempio, prendendo la media) per ottenere una misura migliore delle prestazioni del modello globale. Questa particolare forma di convalida incrociata è una doppia convalida incrociata , ovvero quella in cui abbiamo diviso i dati in due set e li abbiamo utilizzati ciascuno come set di convalida.

Potremmo espandere questa idea per utilizzare ancora più prove e più pieghe nei dati: ad esempio, ecco una rappresentazione visiva della convalida incrociata cinque volte:

![](dati/img/convalida%20incrociata%20cinque%20volte.png)

Qui dividiamo i dati in cinque gruppi e utilizziamo ciascuno di essi a turno per valutare l'adattamento del modello sugli altri 4/5 dei dati. Sarebbe piuttosto noioso farlo a mano, quindi possiamo usare la comoda routine di Scikit-Learn cross_val_score per farlo in modo veloce:

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

Ripetere la validazione su diversi sottoinsiemi di dati ci dà un'idea ancora migliore delle prestazioni dell'algoritmo.

Scikit-Learn implementa una serie di utili schemi di convalida incrociata utili in situazioni particolari; questi sono implementati tramite iteratori nel modulo cross_validation . A esempio, potremmo voler arrivare al caso estremo in cui il nostro numero di pieghe è uguale al numero di punti dati: ovvero, ci alleniamo su tutti i punti tranne uno in ogni prova. Questo tipo di convalida incrociata è noto come convalida incrociata "leave-one-out" e può essere utilizzato come segue:

In [ ]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

Poiché abbiamo 150 campioni, la convalida incrociata "lascia uno fuori" produce punteggi per 150 prove e il punteggio indica una previsione riuscita (1,0) o non riuscita (0,0). Prendendo la media di questi si ottiene una stima del tasso di errore:

In [ ]:
scores.mean()

Altri schemi di convalida incrociata possono essere utilizzati in modo simile.

## Scegliere il modello migliore

Ora che abbiamo visto le basi della validazione e della validazione incrociata, andremo più in profondità per quanto riguarda la selezione del modello e la selezione degli iperparametri. Questi problemi sono alcuni degli aspetti più importanti della pratica dell'apprendimento automatico e trovo che queste informazioni siano spesso trascurate nelle lezioni sull'apprendimento automatico.

Di fondamentale importanza è la seguente domanda: se il nostro stimatore è sottoperformante, come dovremmo andare avanti? Ci sono diverse risposte possibili:

- Utilizzare un modello più complicato/più flessibile
- Utilizzare un modello meno complicato/meno flessibile
- Raccoglire più campioni di formazione
- Raccoglire più dati per aggiungere funzionalità a ciascun campione

La risposta a questa domanda è spesso controintuitiva. In particolare, a volte l'utilizzo di un modello più complicato darà risultati peggiori e l'aggiunta di più campioni di addestramento potrebbe non migliorare i risultati! La capacità di determinare quali passaggi miglioreranno il tuo modello è ciò che distingue i professionisti del machine learning di successo da quelli che non hanno successo.

### Il compromesso Bias- varianza

Fondamentalmente, la questione del “modello migliore” consiste nel trovare un punto debole nel compromesso tra bias e varianza . Considera la figura seguente, che presenta due adattamenti di regressione allo stesso set di dati:

![](dati/img/compromesso%20bias%20varianza.png)

È chiaro che nessuno di questi modelli si adatta particolarmente bene ai dati, ma falliscono in modi diversi.

Il modello a sinistra tenta di trovare un adattamento in linea retta attraverso i dati. Poiché i dati sono intrinsecamente più complicati di una linea retta, il modello a linea retta non sarà mai in grado di descrivere bene questo set di dati. Si dice che un modello di questo tipo non sia adeguato ai dati: cioè non ha una flessibilità sufficiente per tenere adeguatamente conto di tutte le caratteristiche dei dati, un altro modo per dirlo è che il modello ha un bias elevato .

Il modello a destra tenta di adattare un polinomio di ordine superiore attraverso i dati. Qui l'adattamento del modello ha sufficiente flessibilità per tenere conto quasi perfettamente delle caratteristiche fini dei dati, ma anche se descrive in modo molto accurato i dati di addestramento, la sua forma precisa sembra riflettere più le particolari proprietà di rumore dei dati piuttosto che l'intrinseca proprietà di qualunque processo abbia generato tali dati. Come sappiamo, si dice che un modello di questo tipo si adatti eccessivamente ai dati: cioè, ha così tanta flessibilità che il modello finisce per tenere conto degli errori casuali così come della distribuzione dei dati sottostanti, un altro modo per dirlo è che il modello ha una varianza elevata .

Per vedere la cosa sotto un'altra luce, consideriamo cosa succede se utilizziamo questi due modelli per prevedere il valore y di alcuni nuovi dati. Nei diagrammi seguenti, i punti rossi/chiari indicano i dati omessi dal set di addestramento:

![](dati/img/compromesso%20bias%20varianza2.png)

Come sappiamo, il punteggio qui è R2, o coefficiente di determinazione , che misura il rendimento di un modello rispetto a una media semplice dei valori target.
R2= 1 indica una corrispondenza perfetta, R2= 0 indica che il modello non fa niente di meglio che prendere semplicemente la media dei dati, e valori negativi significano modelli ancora peggiori. 
Dai punteggi associati a questi due modelli possiamo fare un’osservazione che vale più in generale:

- Per i modelli con bias elevato, le prestazioni del modello sul set di validazione sono simili alle prestazioni sul set di addestramento.
- Per i modelli ad alta varianza, le prestazioni del modello sul set di validazione sono di gran lunga peggiori delle prestazioni sul set di addestramento.

Se immaginiamo di avere una certa capacità di ottimizzare la complessità del modello, ci aspetteremmo che il punteggio di addestramento e il punteggio di convalida si comportino come illustrato nella figura seguente:

![](dati/img/validation-curve.png)

Il diagramma mostrato qui è spesso chiamato curva di validazione e vediamo le seguenti caratteristiche essenziali:

- Il punteggio di formazione è ovunque superiore al punteggio di convalida. Questo è generalmente il caso: il modello si adatterà meglio ai dati che ha visto che ai dati che non ha visto.
- Per una complessità del modello molto bassa (un modello con bias elevato), i dati di addestramento sono inadeguati, il che significa che il modello è un predittore inadeguato sia per i dati di addestramento che per eventuali dati mai visti in precedenza.
- Per una complessità del modello molto elevata (un modello a varianza elevata), i dati di addestramento sono eccessivamente adattati, il che significa che il modello prevede molto bene i dati di addestramento, ma fallisce per i dati mai visti in precedenza.
- Per alcuni valori intermedi, la curva di validazione ha un massimo. Questo livello di complessità indica un adeguato compromesso tra bias e varianza.

E' importante ricordare che i mezzi per ottimizzare la complessità del modello variano da modello a modello.

### Curve di validazione in Scikit- Learn

Diamo un'occhiata a un esempio di utilizzo della convalida incrociata per calcolare la curva di convalida per una classe di modelli. Qui utilizzeremo un modello di regressione polinomiale, per i parametri del modello $a$ and $b$:

$$
y = ax + b
$$

Un polinomio di grado 3 adatta una curva cubica ai dati; per i parametri del modello $a, b, c, d$:

$$
y = ax^3 + bx^2 + cx + d
$$

Possiamo generalizzare questo a qualsiasi numero di caratteristiche polinomiali. In Scikit-Learn possiamo implementarlo con una semplice regressione lineare combinata con il preprocessore polinomiale. Utilizzeremo una pipeline per mettere insieme queste operazioni :

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

Ora creiamo alcuni dati ai quali adatteremo il nostro modello:

In [ ]:
import numpy as np

def make_data(N, err=1.0, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

X, y = make_data(40)

Ora possiamo visualizzare i nostri dati, insieme agli adattamenti polinomiali di diversi gradi:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


X_test = np.linspace(-0.1, 1.1, 500)[:, None]

plt.scatter(X.ravel(), y, color='black')
axis = plt.axis()
for degree in [1, 3, 5]:
    y_test = PolynomialRegression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label='degree={0}'.format(degree))
plt.xlim(-0.1, 1.0)
plt.ylim(-2, 12)
plt.legend(loc='best')

La manopola che controlla la complessità del modello in questo caso è il grado del polinomio, che può essere qualsiasi numero intero non negativo. Una domanda utile a cui rispondere è questa: quale grado di polinomio fornisce un adeguato compromesso tra bias (sottoadattamento) e varianza (sovraadattamento)?

Possiamo fare progressi in questo visualizzando la curva di validazione per questo particolare dato e modello; questo può essere fatto direttamente utilizzando la comoda routine validation_curve fornita da Scikit-Learn. Dato un modello, dati, nome del parametro e un intervallo da esplorare, questa funzione calcolerà automaticamente sia il punteggio di training che quello di convalida nell'intervallo:

In [ ]:
from sklearn.model_selection import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(
    PolynomialRegression(), X, y,
    param_name='polynomialfeatures__degree',
    param_range=degree, cv=7)

plt.plot(degree, np.median(train_score, 1),
         color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1),
         color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

Ciò mostra esattamente il comportamento qualitativo che ci aspettiamo: il punteggio di formazione è ovunque superiore al punteggio di validazione; il punteggio di addestramento migliora in modo monotono con l'aumentare della complessità del modello; e il punteggio di convalida raggiunge il massimo prima di diminuire quando il modello diventa troppo adatto.

Dalla curva di validazione possiamo leggere che il compromesso ottimale tra bias e varianza si trova per un polinomio di terzo ordine; possiamo calcolare e visualizzare questo adattamento rispetto ai dati originali come segue:

In [ ]:
plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = PolynomialRegression(3).fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim)

Si noti che per trovare questo modello ottimale in realtà non è stato necessario calcolare il punteggio di addestramento, ma esaminare la relazione tra il punteggio di addestramento e il punteggio di convalida può fornirci informazioni utili sulle prestazioni del modello.

## Curve di apprendimento

Un aspetto importante della complessità del modello è che il modello ottimale dipenderà generalmente dalla dimensione dei dati di addestramento. 
Ad esempio, generiamo un nuovo set di dati con un fattore di cinque punti in più:

In [ ]:
X2, y2 = make_data(200)
plt.scatter(X2.ravel(), y2)

Duplicheremo il codice precedente per tracciare la curva di validazione per questo set di dati più grande, per riferimento tracciamo anche i risultati precedenti:

In [ ]:
degree = np.arange(21)
train_score2, val_score2 = validation_curve(
    PolynomialRegression(), X2, y2,
    param_name='polynomialfeatures__degree',
    param_range=degree, cv=7)

plt.plot(degree, np.median(train_score2, 1),
         color='blue', label='training score')
plt.plot(degree, np.median(val_score2, 1),
         color='red', label='validation score')
plt.plot(degree, np.median(train_score, 1),
         color='blue', alpha=0.3, linestyle='dashed')
plt.plot(degree, np.median(val_score, 1),
         color='red', alpha=0.3, linestyle='dashed')
plt.legend(loc='lower center')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score')

Le linee continue mostrano i nuovi risultati, mentre le linee tratteggiate più deboli mostrano i risultati del precedente set di dati più piccolo. È chiaro dalla curva di validazione che il set di dati più grande può supportare un modello molto più complicato: il picco qui è probabilmente intorno al grado 6, ma anche un modello di grado 20 non si adatta seriamente ai dati: nella validazione e addestramento i punteggi rimangono molto vicini.

Vediamo quindi che il comportamento della curva di validazione ha non uno ma due input importanti: la complessità del modello e il numero di punti di addestramento. Spesso è utile esplorare il comportamento del modello in funzione del numero di punti di addestramento, cosa che possiamo fare utilizzando sottoinsiemi di dati sempre più grandi per adattarli al nostro modello. Un grafico del punteggio di addestramento/convalida rispetto alla dimensione del set di addestramento è noto come curva di apprendimento.

Il comportamento generale che ci aspetteremmo da una curva di apprendimento è questo:

- Un modello di una determinata complessità si adatterà eccessivamente a un set di dati di piccole dimensioni: ciò significa che il punteggio di addestramento sarà relativamente alto, mentre il punteggio di convalida sarà relativamente basso.
- Un modello di una determinata complessità non si adatterà a un set di dati di grandi dimensioni: ciò significa che il punteggio di addestramento diminuirà, ma il punteggio di convalida aumenterà.
- Un modello non assegnerà mai, se non per caso, un punteggio migliore al validation set rispetto al training set: questo significa che le curve dovrebbero continuare ad avvicinarsi ma mai a incrociarsi.

Tenendo presenti queste caratteristiche, ci aspetteremmo che una curva di apprendimento assomigli qualitativamente a quella mostrata nella figura seguente:

![](dati/img/learning-curve.png)

La caratteristica notevole della curva di apprendimento è la convergenza verso un punteggio particolare man mano che cresce il numero di campioni di addestramento. 
In particolare, una volta che abbiamo abbastanza punti per far convergere un particolare modello, aggiungere ulteriori dati di addestramento non ti aiuterà! 
L'unico modo per aumentare le prestazioni del modello in questo caso è utilizzare un altro modello (spesso più complesso).

### Curve di apprendimento in Scikit-Learn

Scikit-Learn offre una comoda utilità per calcolare tali curve di apprendimento dai tuoi modelli, ora calcoleremo una curva di apprendimento per il nostro set di dati originale con un modello polinomiale di secondo ordine e un polinomio di nono ordine:

In [ ]:
from sklearn.model_selection import learning_curve

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for i, degree in enumerate([2, 9]):
    N, train_lc, val_lc = learning_curve(
        PolynomialRegression(degree), X, y, cv=7,
        train_sizes=np.linspace(0.3, 1, 25))

    ax[i].plot(N, np.mean(train_lc, 1),
               color='blue', label='training score')
    ax[i].plot(N, np.mean(val_lc, 1),
               color='red', label='validation score')
    ax[i].hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0],
                 N[-1], color='gray', linestyle='dashed')

    ax[i].set_ylim(0, 1)
    ax[i].set_xlim(N[0], N[-1])
    ax[i].set_xlabel('training size')
    ax[i].set_ylabel('score')
    ax[i].set_title('degree = {0}'.format(degree), size=14)
    ax[i].legend(loc='best')

Si tratta di una diagnostica preziosa, perché ci fornisce una rappresentazione visiva di come il nostro modello risponde all'aumento dei dati di addestramento. 
In particolare, quando la curva di apprendimento è già convergente (ovvero, quando le curve di allenamento e di validazione sono già vicine tra loro) l'aggiunta di ulteriori dati di allenamento non migliorerà significativamente l'adattamento! 
Questa situazione è visibile nel pannello di sinistra, con la curva di apprendimento per il modello di grado 2.

L’unico modo per aumentare il punteggio di convergenza è utilizzare un modello diverso (solitamente più complicato). 
Lo vediamo nel pannello di destra: passando a un modello molto più complicato, aumentiamo il punteggio di convergenza (indicato dalla linea tratteggiata), ma a scapito di una maggiore varianza del modello (indicata dalla differenza tra i punteggi di training e di validazione ). 
Se aggiungessimo ancora più dati, la curva di apprendimento per il modello più complicato finirebbe per convergere.

Tracciare una curva di apprendimento per la nostra particolare scelta di modello e set di dati può aiutarci a prendere questo tipo di decisione su come andare avanti nel miglioramento dell'analisi.

## La validazione in pratica: ricerca a griglia

La discussione precedente ha lo scopo di fornire un'intuizione sul compromesso tra bias e varianza e sulla sua dipendenza dalla complessità del modello e dalle dimensioni del set di addestramento. In pratica, i modelli generalmente hanno più di una manopola da girare, e quindi i grafici delle curve di validazione e di apprendimento cambiano da linee a superfici multidimensionali. In questi casi, tali visualizzazioni sono difficili e preferiremmo semplicemente trovare il modello particolare che massimizza il punteggio di validazione.

Scikit-Learn fornisce strumenti automatizzati per eseguire questa operazione nel modulo di ricerca della griglia. Ecco un esempio di utilizzo della ricerca su griglia per trovare il modello polinomiale ottimale. Esploreremo una griglia tridimensionale di caratteristiche del modello; vale a dire il grado del polinomio, il flag che ci dice se adattare l'intercetta e il flag che ci dice se normalizzare il problema. 
Questo può essere impostato utilizzando il meta-stimatore GridSearchCV di Scikit-Learn:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False]}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)

Si noti che, come uno stimatore normale, questo non è stato ancora applicato a nessun dato. 
La chiamata al metodo fit() adatterà il modello a ciascun punto della griglia, tenendo traccia dei punteggi lungo il percorso:

In [ ]:
grid.fit(X, y)

Ora che questo è adatto, possiamo chiedere i parametri migliori come segue:

In [ ]:
grid.best_params_

Infine, se lo desideriamo, possiamo utilizzare il modello migliore e mostrare l'adattamento ai nostri dati utilizzando il codice di prima:

In [ ]:
model = grid.best_estimator_

plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = model.fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim)

La ricerca a griglia offre molte più opzioni, inclusa la possibilità di specificare una funzione di punteggio personalizzata, di parallelizzare i calcoli, di eseguire ricerche casuali e altro ancora.